In [10]:
!pip install langchain_openai
!pip install tiktoken

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import os
from pathlib import Path
import copy
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from langchain_openai import ChatOpenAI
import tiktoken

from collections import defaultdict
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from torch.utils.data import DataLoader, Dataset
from torch import FloatTensor, LongTensor

import sys
sys.path.append('/content/drive/MyDrive/ex2024/')

# colab 関連
from google.colab import files
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = 'NjZiYjBkNjc2MDJlMzIzZmY4MTNlYzg1OlNYWHo1eDdZclpkTkVXRnJLRzNiUGNDRml2YllnSjJB'
os.environ['API_HOST']='https://platform.preferredai.jp'
llm = ChatOpenAI(
    base_url=f"{os.environ['API_HOST']}/api/completion/v1",
    model="plamo-beta",
    streaming=False,
    # other params...,
    max_tokens=2048
)



In [13]:
def load_jsonl(path: Path | str) -> pd.DataFrame:
    path = Path(path)
    return pd.read_json(path, lines=True)


def load_json(path: Path | str) -> dict:
    path = Path(path)
    with path.open() as f:
        data = json.load(f)
    return data

In [14]:
# -*- coding: utf-8 -*-
class Args():
    #BERT訓練済みモデル
    bert_pretrained_model_name = "cl-tohoku/bert-base-japanese-v3"
    max_seq_len = 512 #BERT入力列最大値
    batch_size = 32

    seed: int = 42


    sampling_flag = {'train': True, 'val': False, 'test': False}
    sampling_rate = {'train': 0.35, 'val': 1.0, 'test': 1.0}

    #livedoorニュースコーパスデータセットへのパス
    dataset_dir: Path = Path('/content/drive/MyDrive/ex2024/livedoor/datasets/livedoor/')

    #要約辞書保存先
    summary_dict = defaultdict(lambda: defaultdict(list))
    output_model_dir: Path = Path('/content/drive/MyDrive/ex2024/livedoor/datasets/livedoor/summary/')

In [25]:
def get_summary(batch, batch_size=32):
    res = defaultdict(lambda: defaultdict(list))
    # 使用するモデルに対応するトークナイザを取得
    encoding = tiktoken.get_encoding("cl100k_base")

    # 最大トークン数を設定（例: 4096 - 出力用トークンを残して 2048 トークン）
    input_max_tokens = 1700
    for i in range(batch_size):
        input_tokens = encoding.encode(batch['body'][i])
        num_tokens = len(input_tokens)
        body = batch['body'][i]
        if num_tokens > input_max_tokens:
            # トークンを制限内に収めるためにカット
            input_tokens = input_tokens[:input_max_tokens]
            # トークンを再びテキストに変換
            trimmed_prompt = encoding.decode(input_tokens)
            body = trimmed_prompt
        messages=[
            {"role": "system", "content": "あなたは優秀なAIアシスタントです。"},
            {"role": "user", "content": "以下は「{0}」というタイトルの記事の本文です。内容を要約し、結果のみを出力してください。".format(batch['title'][i])},
            {"role": "user", "content": "{0}".format(body)}
        ]


        res[batch['category-ids'][i]] = llm(messages).content
        print(batch['title'][i][:100])
        print("---------------")
        print(res[batch['category-ids'][i]][:100])
        print("===============")
    return res


In [33]:
class Experiment():
    def __init__(self, _args: Args):
        #実験設定
        self.args = _args
        self.phase = ['train_1','train_2','train_3','train_4']
        #データローダー
        self.dataloader = defaultdict(lambda: defaultdict(list))
        #データセット(DataFrame)
        self.dataset = defaultdict(lambda: defaultdict(list))
        for p in self.phase:
            self.dataloader[p] = self.load_dataset(phase = p)


    def load_dataset(self, phase):
        path = self.args.dataset_dir / f'{phase}.jsonl'
        original = load_jsonl(path)

        data = original.to_dict(orient="records")
        self.dataset[phase] = pd.DataFrame(data.copy())
        return DataLoader(data, collate_fn=self.collate_fn, batch_size=self.args.batch_size,shuffle=False,num_workers=2,pin_memory=True)

    # カスタムcollate_fnの定義
    def collate_fn(self, batch):
        title = [data['title'] for data in batch]
        body = [data['body'] for data in batch]
        #category-idから元のデータを参照できるようにする
        category_ids = [data['category-id'] for data in batch]
        return {'title': title, 'body': body, 'category-ids': category_ids}

    def category_id2article(self, phase, category_id):
        return self.dataset[phase][self.dataset[phase]['category-id'] == category_id]


    def train(self):
        for phase in ['train_1']:
            print(phase + " start!!\n")
            dataloader_size = len(self.dataloader[phase])
            cnt = 0
            for batch in self.dataloader[phase]:
                print("{}/{}\n".format(cnt,dataloader_size))
                print(batch['title'])
                print(len(batch['title']))
                cnt = cnt + 1
                self.args.summary_dict.update(get_summary(batch, len(batch['title'])))

        # JSONファイルに保存
        with open('/content/drive/MyDrive/ex2024/livedoor/datasets/livedoor/summary/livedoor_summary_plamo_beta_train_1.json', 'w') as json_file:
            json.dump(self.args.summary_dict, json_file)

    def save_model(self):
        torch.save(self.net.state_dict(), self.new_model_path)
        print('\nbest score updated, Pytorch model was saved!! val_acc:{}\n'.format(self.history.best['val']['acc']))


    def run(self):
        print("ex start")
        self.label_distribution()
        self.train()

    def label_distribution(self):
        # ラベルごとのデータ数を集計
        for p in self.phase:
            label_counts = self.dataset[p]['label'].value_counts()
            print(p, label_counts)

In [34]:
ex = Experiment(_args = Args())
ex.run()
print('---finish---')

Streaming output truncated to the last 5000 lines.

このスマートフォンは高性能CPUを搭載し、HD動画
Peachyアプリアンケートへのご協力、ありがとうございました!
---------------
 Peachyアプリのユーザーアンケートの結果、多くのユーザーが記事の増加を希望し、特に恋愛コラム、美容、レシピ、ダイエット、占いに関する内容を求めていることが分かりました。また、アプリのバグや強制終
iPhoneで作るビジネス文書、動画のムダをトリミング iPhone活用術【知っ得!虎の巻】
---------------
 与えられた文章を要約します。

iPhoneをビジネスに活用する方法が掲載された記事の要約は以下の通りです。

・子どもがiPhoneを使う際、アプリの機能制限を設定するなど、いたずらや危険なサイト
いよいよ明日!JAXAが行う「みんなで皆既月食を観察しようキャンペーン」【話題】
---------------
 要約:

日本のJAXA宇宙教育センターが「みんなで皆既月食を観察しようキャンペーン」を開始し、明日の皆既月食の観測写真やスケッチを募集しています。太平洋側では晴天が期待され、絶好の観察日和となりそ
「メガロス」1日無料体験チケットをブロガー限定プレゼント
---------------
 メガロスがブロガー限定で、施設の1日無料体験チケットをプレゼントしている。メガロスは、関東を中心とした国内最大級のスポーツクラブで、設備やスタッフの対応に定評がある。夏が近づいているこの機会に、実際
第24回東京国際映画祭作品ガイド<香川京子と巨匠たち部門>
---------------
 香川京子と巨匠たち部門で上映される作品の一覧です。各作品の内容を簡潔にまとめました。

「近松物語」：京都の大店の手代と奉公人の不義密通をめぐる悲恋物語を、巨匠・溝口健二が映画化した傑作。

「王将
猫ひろしに負けたカンボジア選手に「何も言う資格無い」との声も
---------------
 この記事では、カンボジア国籍を取得しロンドンオリンピックにカンボジア代表として出場した猫ひろしに対する批判の声がネット掲示板や著名人から上がっていると報じています。主な批判内容は、猫ひろしがカンボジ
毎日の生活の中でお